In [1]:
import numpy as np
import pandas as pd

In [2]:
my_df_1 = pd.DataFrame({0: [0, 0, 1, 1]*2, 1: [1, 0, 1, 0]*2, 2: [1, 1, 1, 1, 0, 0, 0, 0], 3: [123]*8})
my_df_1.to_csv("my_df_1.csv", header=None, index=None)
my_df_1

,0,1,2,3
0,0,1,1,123
1,0,0,1,123
2,1,1,1,123
3,1,0,1,123
4,0,1,0,123
5,0,0,0,123
6,1,1,0,123
7,1,0,0,123


In [3]:
np.random.seed(123)
my_df_2 = pd.DataFrame({0: [0, 1], 1: [1, 0], 2: [0, 1], 3: [123]*2})
my_df_2

,0,1,2,3
0,0,1,0,123
1,1,0,1,123


In [4]:
pd.concat([my_df_1, my_df_2]).to_csv("my_df_12.csv", header=None, index=None)

In [5]:
import csv
import numpy as np
from scipy.stats import rankdata
from pandas import read_csv, DataFrame
from itertools import product, chain, starmap, combinations, combinations_with_replacement
from time import time

time0 = time()

k = 2
divisions = 1
range_ = 0.00
seed = 123

# 1. Function definitions

def discretize(seq, divisions=divisions, range_=range_, seed=seed):
    '''
    >>> discretize([3, 4, 1, 8, 13, 8], divisions=4, range_=0, seed=123) = array([1, 1, 0, 2, 3, 2])
    where
    ranks = [2., 3., 1., 4.5, 6., 4.5]
    tresholds = [1.5,  3.,  4.5]
    '''
    np.random.seed(seed)
    ranks = rankdata(seq, method='ordinal') # method='ordinal'/'average' ?
    #print(1, ranks)
    random_blocks = np.cumsum(range_ * (2 * np.random.random(divisions + 1) - 1) + np.ones(divisions + 1))
    tresholds = random_blocks[:-1] / random_blocks[-1] * len(seq)
    discrete_seq = np.zeros(len(seq), dtype='float64')
    #print(2, tresholds)
    for treshold in tresholds:
        discrete_seq[ranks > treshold] += 1
    #print(3, discrete_seq)
    return discrete_seq

discretize_vec = np.vectorize(discretize, signature='(n)->(n)', excluded=['divisions', 'range_', 'seed'])

# Readthe data

file = "my_df_12.csv"
data = []
with open(file) as csvfile:
    reader = csv.reader(csvfile, delimiter=',',
                        quoting=csv.QUOTE_NONNUMERIC)
    for row in reader:
        data.append(row)
        
data = np.array(data, dtype='float64').T[:-1]
#print(data)
data[:-1] = discretize_vec(data[:-1])
#print(data)
data = data.astype('int64')
labels, counts = np.unique(data[-1], return_counts=True)
label_counts = {int(label): label_count for (label, label_count) in zip(labels, counts)}
min_count = np.min(counts)

dim0, dim1 = data[:-1].shape 
n_classes = len(np.unique(data[-1]))

#print("dims:", dim0, dim1, ", number of classes:", n_classes, ", k=", k)

# 2. Function definitions

def jobs_generator(k=k, dim0=dim0):
    '''
    Python-generator.
    E.g. output for k=2:
    {0,1}, {0,2}, ..., {0, dim0-1}, {1,2}, ..., {1,dim0-1}, ..., {dim0-2, dim0-1}
    Go with combinations(range(M), k) to exclude diagonal tuples
    '''        
    return combinations(range(dim0), k)    

def neg_H(p):
    return p * np.log2(p)

def neg_H_cond(matrix):
    return np.sum(neg_H(matrix)) - np.sum(neg_H(np.sum(matrix, axis=-1)))

xi = 1e-10
def work(indeces):
    '''
    Work-function.
    Output: {indexA: (number, list-of-indeces), indexB: ..., ...}
    indeces -> tuple
    '''
    # contingency-matrix: begin with pseudo-counts
    contingency_m = np.ones([divisions + 1] * k + [len(labels)], dtype='float64')
    for label, count in label_counts.items():
        contingency_m[..., label] *= xi * (count / min_count)
    
    # contingency-matrix: normal counts
    for c_index in data[list(indeces) + [-1]].T:
        #print(c_index)
        contingency_m[tuple(c_index)] += 1
        #print(contingency_m)
    
    results = {}
    for i, index in enumerate(indeces):
        #print(i, np.sum(contingency_m, axis=i))
        result = neg_H_cond(contingency_m) - neg_H_cond(np.sum(contingency_m, axis=i))
        results[index] = (result, indeces)
    return results


def record(results, records):
    '''
    results, records -> dicts
    Accepts output of the work-function and updates the dict that accumulates global results
    '''
    for index, score in results.items():
        if index not in records or score[0] > records[index][0]:
            records[index] = score


final_results = {}

for job in jobs_generator():
    results = work(job)
    record(results, final_results)

# result
print("Finished in", time()-time0, "sec.")
DataFrame(final_results).T.rename(columns={0: 'IG_max', 1: 'tuple'})

Finished in 0.002930879592895508 sec.


,IG_max,tuple
0,0.199731,"(0, 1)"
1,0.199731,"(0, 1)"


### R

install.packages("MDFS")

my_df = read.csv("my_df_1.csv", header=FALSE)

MDFS::ComputeMaxInfoGains(my_df[,1:2], my_df[,3], dimensions=2, divisions = 1, range=0, pseudo.count=1e-10, seed=123)
